# -------------------- EDA JUAN ZUBIAGA ------------------------

## 0. IMPORTAR LIBRERIAS Y DATOS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
from bs4 import BeautifulSoup as bs
import requests
import pickle

pd.set_option('max_colwidth', 800)

In [ ]:
movdat="/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/J-EDA/TMDB_Movie_Limpito.csv"
moviesdbinfo=pd.read_csv(movdat)
moviesdb=moviesdbinfo.drop(columns={'overview'})

In [ ]:
mov322= "/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/J-EDA/100movies322.csv"

movies322= pd.read_csv(mov322)
movies322.columns

He tenido un desajuste con otra de las pestañas en las que he desarrollado una de las hipótesis, entonces tengo que hacer una copia de movies322 quitando alguna de las columnas

In [ ]:
movies22=movies322[['Title','IMDB Rating','Year','Runtime','Genre','Budget','Income','Country_of_origin']]

In [ ]:
movi= "/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/J-EDA/movies.csv"
movies= pd.read_csv(movi)
movies.columns


In [ ]:
oscar2="/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/J-EDA/the_oscar_award.csv"
oscar2=pd.read_csv(oscar2)
oscar2.head()

In [ ]:
os= "/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/oscars_df.csv"
oscars= pd.read_csv(os)
oscars[oscars['Award']=='Winner']

No he encontrado informacion fiable sobre los Premios Goya, por lo que he recopilado manualmente a través de diversas fuentes (principalmente Wikipedia), todas las ganadoras a Mejor Película.

In [ ]:
conteo_goya= pd.Series([1,4,11,26,8,2,2], 
                       index=['Supervivencia', 'Suspense', 'Comedia','Drama', 'Thriller', 'Fantasía','Terror'])

# 1. LIMPIAR DATOS

## 1.1

In [ ]:
movies22.dtypes

In [ ]:
movies22.isnull().sum()

Al intentar convertir las variables numericas a ints o floats para combinarla con otra ds me daba error porque existen unknowns. Tengo que convertirlos a Nans para poder continuar.

In [ ]:
movies22.replace('Unknown',pd.NA,inplace=True)
movies22.isnull().sum()

In [ ]:
movies22[(movies22['Runtime'].isna())|(movies22['IMDB Rating'].isna())]

Ya que mi hipotesis se centra en películas minimamente populares o conocidas, voy a optar por ignorar de mi estudio Mantus, que no tiene ni pagina en IMDB, y My Little Princess, que es un cortometraje con una valoración muy baja. Y a cambio de este ahorro en esfuerzo, voy a completar la informacion de la primera, que si conozco, con webscrapping, en lugar de buscarla en el movil y rellenarla manualmente.

In [ ]:
movies22.drop(labels=[848,1249],inplace=True)

In [ ]:
from fake_useragent import UserAgent
ua= UserAgent()
headers = {'User-Agent': ua.random}
url= "https://www.imdb.com/es-es/title/tt7405458/"
response = requests.get(url, headers=headers)
print(response)
html=response.content
soup = bs(html, 'html.parser')

In [ ]:
ottorating= soup.findAll('span',class_='sc-d541859f-1 imUuxf')
ottorating[0].text

In [ ]:
movies22['IMDB Rating']=movies22['IMDB Rating'].fillna(ottorating[0].text)
movies22.iloc[84:87]

In [ ]:
movies22['IMDB Rating']=movies22['IMDB Rating'].replace('7,5','7.5')
movies22.iloc[84:87]

In [ ]:
movies[movies['original_title']!=movies['title']]

Veo que el original title es el titulo en el idioma de producción, por lo que para el estudio solo me interesa el titulo en inglés, que coincide en su mayoria con el original. Lo siguiente es quedarme solo con el año en la fecha de estreno y tratar de filtrarla.

In [ ]:
movies.drop(columns=['original_title','homepage','id','keywords','original_language','overview','production_companies','spoken_languages','status','tagline','vote_average', 'vote_count','cast','crew','director'],inplace=True)
movies.dtypes

In [ ]:
movies[movies['popularity']<2]

In [ ]:
print(movies[movies['popularity']<1.5].isnull().sum())
print(movies[movies['popularity']>1.5].isnull().sum())

Por la misma razon, voy a descartar aquellas con indices de popularidad bajos, que además resultan ser en las que se concentran los NaNs.

In [ ]:
movies.drop(movies[movies['popularity']<1.5].index,inplace=True)
movies.shape

In [ ]:
movies.rename(columns={'release_date':'Year'},inplace=True)
movies.rename(columns={'title':'Title'},inplace=True)
movies['Year'] = movies['Year'].apply(lambda x: str(x).split('-')[0])
movies.head()


Para poder comparar las duraciones de las dos ds tengo que hacer que las variables que voy a juntar sean del mismo tipo.

In [ ]:
movies.dtypes

In [ ]:
movies22['Runtime']=movies22['Runtime'].fillna(0)
movies22['Runtime']=movies22['Runtime'].astype('Int64')
movies22['Runtime']=movies22['Runtime'].replace(0,pd.NA)
movies['runtime']=movies['runtime'].fillna(0)
movies['runtime']=movies['runtime'].astype('Int64')
movies['runtime']=movies['runtime'].replace(0,pd.NA)
movies.rename(columns={'runtime':'Runtime'},inplace=True)
movies['Year']=movies['Year'].fillna(0)
movies['Year']=movies['Year'].astype(int)
movies['Year']=movies['Year'].replace(0,pd.NA)
movies['popularity']=movies['popularity'].fillna(0)
movies['popularity']=movies['popularity'].astype(float)
movies['popularity']=movies['popularity'].replace(0,pd.NA)
movies22['IMDB Rating']=movies22['IMDB Rating'].fillna(0)
movies22['IMDB Rating']=movies22['IMDB Rating'].astype(float)
movies22['IMDB Rating']=movies22['IMDB Rating'].replace(0,pd.NA)

In [ ]:
print(movies.dtypes)
print(movies22.dtypes)

In [ ]:
print(movies22.isnull().sum())
print(movies.isna().sum())


In [ ]:
movies[movies['Runtime'].isna()]

He estado indagando y preguntando en los foros y a copilot y no consigo entender porque me crea estos nans, pero por suerte es en peliculas poco conocidas que en un ds tan grande puedo ignorar.

In [ ]:
movies.drop(labels=[3809,4241],inplace=True)

In [ ]:
print(movies22.isnull().sum())
print(movies.isna().sum())


In [ ]:
moviesdbinfo.isnull().sum()

In [ ]:
moviesdb.rename(columns={'release_date':'Year'},inplace=True)
moviesdb.rename(columns={'title':'Title'},inplace=True)
moviesdb['popularity'] = moviesdb['popularity'].apply(lambda x: str(x).split('.')[0])
moviesdb['popularity'] = moviesdb['popularity'].astype(int)
moviesdb['Year'] = moviesdb['Year'].apply(lambda x: str(x).split('-')[0])
moviesdb['Year']=moviesdb['Year'].replace('nan','0')
moviesdb['Year']=moviesdb['Year'].fillna(0)
moviesdb['Year']=moviesdb['Year'].replace(0,pd.NA)
moviesdb['Year']=moviesdb['Year'].astype('int')
print(moviesdb.dtypes)
moviesdb

In [ ]:
moviesdb.isnull().sum()

He encontrado una base de datos de un millon de peliculas y a primera vista bastante limpia. Ya la he reducido en otra ventana quitando columnas que no me interesan, documentales, peliculas cortas y peliculas para adultos, para poder importarla mas rapido, y ahora tengo que filtrarla bastante tratando de obtener peliculas populares y minimamente votadas. Que sea tan extensa me baja el riesgo de que un error de sus escritores me afecte mucho al analisis. 

In [ ]:
moviesdb[(moviesdb['Title'].str.contains('sex'))|(moviesdb['Title'].str.contains('Sex'))]

In [ ]:
moviesdb[((moviesdb['Title'].str.contains('sex'))|(moviesdb['Title'].str.contains('Sex')))&((moviesdb['vote_count']<10))&(moviesdb['popularity']<5)]

In [ ]:
mascarasx= ((moviesdb['Title'].str.contains('sex', case=False))&(((moviesdb['vote_count']<10)&(moviesdb['popularity']<5))|(moviesdb['budget']==0)))
moviesdb=moviesdb[~mascarasx]

In [ ]:
moviesdb[moviesdb['Title'].astype(str).str.contains('Sex',case=False)].head(10)

Aún sigue habiendo peliculas con títulos extraños, asi que tengo que filtrar por un minimo de probabilidad o nº de votos.
He visto que no puedo filtrar por recaudacion, lo que me seria muy facil para distinguir las peliculas de salas de cine, porque se incluyen multitud de peliculas que se estrenaron directamente en las plataformas de streaming y no recaudaron nada en salas.

In [ ]:
oscar2['category']=oscar2['category'].replace(['OUTSTANDING PICTURE','OUTSTANDING PRODUCTION','OUTSTANDING MOTION PICTURE','BEST MOTION PICTURE'],'BEST PICTURE')


-------------------------

## ANÁLISIS ESTADÍSTCO
A continuación, queda estudiar los datos tratando de encontrar otros outliers y valores no esperados que puedan afectar a los análisis y representaciones posteriores.

In [ ]:
movies322['Income']=movies322['Income'].replace('Unknown',pd.NA)
movies322['Income']=movies322['Income'].str.replace('[$,]','',regex=True).str.strip()
movies322['Income']=pd.to_numeric(movies322['Income'],errors='coerce')
movies322['Income'].median()

In [ ]:
Q1 = movies322['Income'].quantile(0.25)
Q3 = movies322['Income'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identificar los valores atípicos
ioutliers = movies322[(movies322['Income'] < lower_bound) | (movies322['Income'] > upper_bound)]
ioutliers.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

sns.boxenplot(x=movies322['Income'], ax=axes[0], color="blue")
axes[0].set_title("Distribución")
axes[0].set_xlabel("Recaudación")

sns.boxenplot(x=movies322['Budget'], ax=axes[1], color="green")
axes[1].set_title("Distribución")
axes[1].set_xlabel("Presupuesto")

In [ ]:
movies322[(movies322['Income']>2000000000)|(movies322['Income']<11000)].sort_values('Income').head()

Una vez comprobados los datos de presupuesto y recaudacion de esta ds, no hay ningun valor atípico (mas allá de las peliculas mas caras y las que han tenido recaudaciones record, y las peliculas de producción 'casera' y poco conocidas). Sin embargo, el csv de db con un millon de datos lo enccontré después del estudio y limpieza de movies y movies322, y tras haberla contrastado, decidí centrarme en db pues tenia datos igual de fiables de miles de películas mas. 
En el notebook llamado Movies puedes encontrar una parte del análisis y limpieza que hice de las otras ds.

In [ ]:
Q1 = moviesdb['revenue'].quantile(0.25)
Q3 = moviesdb['revenue'].quantile(0.75)
IQR = Q3 - Q1

liminf = Q1 - 1.5 * IQR
limsup = Q3 + 1.5 * IQR

# Identificar los valores atípicos
dbrevoutliers = moviesdb[(moviesdb['revenue'] < liminf) | (moviesdb['revenue'] > limsup)]
dbrevoutliers.describe()

In [ ]:
moviesdb['revenue'].describe(percentiles=[.25, .5, .75, .9, .99])

In [ ]:
Q1 = moviesdb['runtime'].quantile(0.25)
Q3 = moviesdb['runtime'].quantile(0.75)
IQR = Q3 - Q1

run_liminf = Q1 - 1.5 * IQR
run_limsup = Q3 + 1.5 * IQR

# Identificar los valores atípicos
dbrunoutliers = moviesdb[(moviesdb['runtime'] < run_liminf) | (moviesdb['runtime'] > run_limsup)]
dbrunoutliers.describe()

In [ ]:
moviesdb['runtime'].describe(percentiles=[.25, .5, .75, .9, .99])

In [ ]:
Q1 = moviesdb['budget'].quantile(0.25)
Q3 = moviesdb['budget'].quantile(0.75)
IQR = Q3 - Q1

bud_liminf = Q1 - 1.5 * IQR
bud_limsup = Q3 + 1.5 * IQR

# Identificar los valores atípicos
dbrunoutliers = moviesdb[(moviesdb['budget'] < bud_liminf) | (moviesdb['budget'] > bud_limsup)]
dbrunoutliers.describe()

In [ ]:
moviesdb['budget'].describe(percentiles=[.25, .5, .75, .9, .99])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
sns.boxenplot(x=moviesdb['runtime'], ax=axes[1], color="blue")
plt.title('Distr')
axes[1].set_title("Distribución de la duración")
axes[1].set_xlabel("Duración")

sns.boxenplot(x=moviesdb['revenue'], ax=axes[0], color="green")
axes[0].set_title("Distribución de la recaudación")
axes[0].set_xlabel("Recaudación")


In [ ]:
sns.boxenplot(x=moviesdb['budget'], color="red")
plt.xlabel('Presupuesto')
plt.title('Distribución del presupuesto')

Me presentan boxplots muy raros pero no sorprenden, pues incluye películas que no recaudaron nada, y otras que recaudaron mas de 3000 millones, y lo mismo en el presupuesto, con películas de produccion casera y peliculas de presupuesto record. 

In [ ]:
moviesdb.sort_values('runtime',ascending=False).head()

In [ ]:
moviesdb[(moviesdb['runtime']>240)&(moviesdb['popularity']<=11)]


In [ ]:
mascaralargas= ((moviesdb['runtime']>240)&(moviesdb['popularity']<=10))
moviesdb=moviesdb[~mascaralargas]

In [ ]:
moviesdb['popularity'].describe(percentiles=[.25, .5, .75, .95, .99])

In [ ]:
moviesdb[(moviesdb['popularity']==0)&(moviesdb['revenue']>0)].sort_values('popularity',ascending=False)

In [ ]:
moviesdb=moviesdb[(moviesdb['vote_count']>1)|((moviesdb['popularity']==0)&(moviesdb['revenue']>0))]
print(moviesdb.shape)
moviesdb['popularity'].describe(percentiles=[.25, .5, .75, .95, .99])

In [ ]:
moviesdb[moviesdb['popularity']>500].sort_values('popularity',ascending=False).head(5)

Por lo que veo, la popularidad no es muy objetivo (y por tanto fiable a nivel cuantitativo) pues me extraña que, estando el DS ordenado por numero de votos, entre las peliculas con mayor popularidad no haya ninguna de las 1000 que mas votos han recibido, y haya tantas a priori muy poco conocidas y de "mercados" pequeños como Nigeria o Islandia. Para tratar de aumentar su fiabilidad, voy a modificar los valores superiores al percentil 99 para tratar de equilibrarla un poco.

In [ ]:
moviesdb.loc[moviesdb['popularity'] > 1000, 'popularity'] = moviesdb['popularity'] / 10


In [ ]:
moviesdb['popularity'].describe()

Incluso con la modificacion carece de utilidad pues ademas de lo observado, el percentil 50 no sube del valor 1. Me ha valido para quitarme los volúmenes y sagas agrupadas que se incluian como películas, pero no voy a darle mas uso.

Tampoco me puedo fiar por presupuesto pues a tiene algunos errores y valores no esperados, como películas de grandes estudios a las que asigna un presupuesto de 0$. El CSV esta ordenado por nº de votos por lo que voy a probar con esa variable.

In [ ]:
print(moviesdb.shape)
moviesdb['vote_count'].describe(percentiles=[.10,.25,.5,.6,.85,.9,.95,.98])

In [ ]:
moviesdb[(moviesdb['vote_count']<66)].sort_values('vote_count',ascending=False).head(10)

In [ ]:
conteo_generosnom=oscars['Movie Genre'].str.split(',').explode().replace({
    'Musical': 'Música',
    'Music':'Música',
    'Film-Noir': 'Crimen',
    'Comedy':'Comedia',
    'Crime':'Crimen',
    'War': 'Acción',
    'Adventure':'Acción',
    'Family':'Familiar',
    'Action':'Acción',
    'Mystery':'Thriller',
    'History':'Historia/Bio',
    'Biography': 'Historia/Bio',
    'Fantasy':'Fantasia',
    'Sport':'Deporte',
    'Horror':'Otros',
    'Animation':'Otros',
    }).value_counts()


In [ ]:
oscarwinners=oscars[oscars['Award']=='Winner']
conteo_generosw=oscarwinners['Movie Genre'].str.split(',').explode().replace({
    'Musical': 'Música',
    'Music':'Música',
    'Film-Noir': 'Crimen',
    'Comedy':'Comedia',
    'Crime':'Crimen',
    'War': 'Acción',
    'Adventure':'Acción',
    'Family':'Familiar',
    'Action':'Acción',
    'Mystery':'Thriller',
    'History':'Historia/Bio',
    'Biography': 'Historia/Bio',
    'Fantasy':'Fantasia',
    'Sport':'Deporte',
    }).value_counts()

In [ ]:
oscarwinners1=oscarwinners.copy()
oscarwinners1['Movie Genre']=oscarwinners['Movie Genre'].apply(lambda x: x.split(',')[0])
conteo_oscargenprincipal=oscarwinners1['Movie Genre'].value_counts()
conteo_oscargenprincipal

Como el csv de los Oscar lo he obtenido de la Academia no voy a limpiarla en exceso pues es una fuente fiable y no voy a atender a las variables numericas sino unicamente el tipo de galardón y el nombre, género y año de la película. Así, la unica limpieza o modificación que he hecho es separar los generos de forma individual y quedarme con el principal en una de ellas, para hacer diferentes graficos.

-----

Para la representacion y la creación de graficos voy a crear diferentes mascaras de db para poder exponer diferentes gráficos, con por ejemplo, las peliculas populares que mas han duran, las peliculas españolas que mas han recaudado, etc.

In [ ]:
movies_=pd.merge(movies,movies22,on=['Title','Year'],how='outer')
movies_['Runtime_x']=movies_['Runtime_x'].fillna(movies_['Runtime_y'])
movies_['genres']=movies_['genres'].fillna(movies_['Genre'])
movies_.drop(columns={'Runtime_y','Genre'},inplace=True)


In [ ]:
movies_largas=movies_[(movies_['Runtime_x']>180)&((movies['popularity']>8)|(movies_['IMDB Rating']>3))]
movies_largas.sort_values('Year',ascending=False).head()


In [ ]:
print(moviesdb.shape)
print(moviesdb.isnull().sum())
print(movies_.shape)
print(movies_.isnull().sum())

In [ ]:
print(moviesdb.shape)
print(moviesdb[moviesdb['revenue']==0].shape)
print(moviesdb[moviesdb['runtime']==0].shape)
print(moviesdb[moviesdb['budget']==0].shape)

In [ ]:
moviesdb_=pd.merge(moviesdb,movies_, on=['Title','Year'], how='outer')
print(moviesdb_.shape)
moviesdb_.head()

In [ ]:
moviesdb_['revenue_x']=moviesdb_['revenue_x'].replace(0,pd.NA)
moviesdb_['revenue_x']=moviesdb_['revenue_x'].fillna(moviesdb_['revenue_y'])
moviesdb_['runtime']=moviesdb_['runtime'].replace(0,pd.NA)
moviesdb_['runtime']=moviesdb_['runtime'].fillna(moviesdb_['Runtime_x'])
moviesdb_['budget_x']=moviesdb_['budget_x'].replace(0,pd.NA)
moviesdb_['budget_x']=moviesdb_['budget_x'].fillna(moviesdb_['budget_y'])
moviesdb_['genres_x']=moviesdb_['genres_x'].fillna(moviesdb_['genres_y'])
moviesdb_['vote_average']=moviesdb_['genres_x'].fillna(moviesdb_['IMDB Rating'])
moviesdb_['popularity_x']=moviesdb_['popularity_x'].replace(0,pd.NA)
moviesdb_['popularity_x']=moviesdb_['popularity_x'].fillna(moviesdb_['popularity_y'])
moviesdb_['production_countries_x']=moviesdb_['production_countries_x'].fillna(moviesdb_['production_countries_y'])
moviesdb_.drop(columns=['revenue_y','budget_y','Runtime_x','popularity_y','production_countries_y','genres_y','index','IMDB Rating'],inplace=True)
print(moviesdb_.isnull().sum())
print(moviesdb_.shape)

In [ ]:
print(moviesdb_.shape)
print(moviesdb_[moviesdb_['revenue_x']==0].shape)
print(moviesdb_[moviesdb_['runtime']==0].shape)
print(moviesdb_[moviesdb_['budget_x']==0].shape)

In [ ]:
moviesdb_['decade'] = (moviesdb_['Year'] // 10 * 10).astype(int).astype(str) + "s"
moviesdb_['decade_numeric'] = moviesdb_['Year'] // 10 * 10


In [ ]:
moviesdb[moviesdb['runtime']>180]
db180p=moviesdb_[(moviesdb_['runtime']>180)&(moviesdb_['popularity_x']>20)]
dbpp=moviesdb_[(moviesdb_['vote_count']>133)]
dbpop=moviesdb_[(moviesdb_['vote_count']>1000)]
dbesp=moviesdb_[moviesdb_['production_countries_x'].astype(str).str.contains('Spain')]
dbpop['Year'].value_counts().sort_index().index

In [ ]:
dbpop.loc['decade'] = (dbpop['Year'] // 10 * 10).astype(int).astype(str) + "s"
dbpop.loc['decade_numeric'] = dbpop['Year'] // 10 * 10

In [ ]:
moviesdb.shape

In [ ]:
dbpop.sort_values('runtime',ascending=False).head(10)

In [ ]:
plt.figure(figsize=(8,8))
sns.lineplot(dbpp.groupby('Year')['runtime'].mean(),color='green')
plt.title('DURACION MEDIA DE LAS PELICULAS DE CADA AÑO')
plt.ylabel('DURACIÓN (MINUTOS)')
plt.xlabel('AÑO')
plt.xticks(ticks=range(1922,2032,10))
plt.show()

In [ ]:
dbpop.groupby('Year')['runtime'].max().sort_values().index[-2]

In [ ]:
dbpmax = dbpp['runtime'].max()
dbpmaxyear = dbpp[dbpp['runtime'] == dbpmax]['Year'].values[0]
dbpmaxtitle = dbpp[dbpp['runtime'] == dbpmax]['Title'].values[0]
dbpmin = (dbpp.groupby('Year')['runtime'].max()).min()
dbpminyear = dbpp[dbpp['runtime'] == dbpmin]['Year'].values[0]
dbpmintitle = dbpp[dbpp['runtime'] == dbpmin]['Title'].values[0]
dbp2max = dbpp['runtime'].nlargest(2).iloc[-1]
dbp2maxyear = dbpp[dbpp['runtime'] == dbp2max]['Year'].values[0]
dbp2maxtitle = dbpp[dbpp['runtime'] == dbp2max]['Title'].values[0]
dbp3max = dbpp['runtime'].nlargest(3).iloc[-1]
dbp3maxyear = dbpp[dbpp['runtime'] == dbp3max]['Year'].values[0]
dbp3maxtitle = dbpp[dbpp['runtime'] == dbp3max]['Title'].values[0]


In [ ]:
plt.figure(figsize=(10,10))
sns.lineplot(dbpp.groupby('Year')['runtime'].max())
plt.title('DURACIÓN DE LAS PELICULAS MAS LARGAS DE CADA AÑO \n (MINIMAMENTE CONOCIDAS)')
plt.ylabel('DURACIÓN (MINUTOS)')
plt.xlabel('Año')
plt.xticks(ticks=range(1911,2031,10))
plt.axhline(y=180, color='red', linestyle='dashdot',)
plt.axhline(y=120, color='orange', linestyle='dashdot')
plt.axhline(y=240, color='maroon', linestyle='dashdot')
plt.axhline(y=300, color='black', linestyle='dashdot')
plt.text(dbpmaxyear, dbpmax, f'{dbpmaxtitle}, {int(dbpmax)} min.', horizontalalignment='right')
plt.plot(dbpmaxyear, dbpmax, 'ro')
plt.text(dbp2maxyear, dbp2max, f'{dbp2maxtitle}, {int(dbp2max)} min.', horizontalalignment='right')
plt.plot(dbp2maxyear, dbp2max, 'go')
plt.text(dbp3maxyear, dbp2max, f'{dbp2maxtitle}, {int(dbp2max)} min.', horizontalalignment='right')
plt.plot(dbp3maxyear, dbp2max, 'go') 
plt.text(dbpminyear, dbpmin, f'{dbpmintitle}, {int(dbpmin)} min.', horizontalalignment='right')
plt.plot(dbpminyear, dbpmin, 'bo')  
plt.show()

In [ ]:
dbpop.groupby('Year')['runtime'].max().sort_values().head(-1)

In [ ]:
dbmax = dbpop['runtime'].max()
dbmaxyear = dbpop[dbpop['runtime'] == dbmax]['Year'].values[0]
dbmaxtitle = dbpop[dbpop['runtime'] == dbmax]['Title'].values[0]
dbmin = (dbpop.groupby('Year')['runtime'].max()).min()
dbminyear = dbpop[dbpop['runtime'] == dbmin]['Year'].values[0]
dbmintitle = dbpop[dbpop['runtime'] == dbmin]['Title'].values[0]
db2max = dbpop['runtime'].nlargest(2).iloc[-1]
db2maxyear = dbpop[dbpop['runtime'] == db2max]['Year'].values[0]
db2maxtitle = dbpop[dbpop['runtime'] == db2max]['Title'].values[0]
db3max = dbpop['runtime'].nlargest(3).iloc[-1]
db3maxyear = dbpop[dbpop['runtime'] == db3max]['Year'].values[0]
db3maxtitle = dbpop[dbpop['runtime'] == db3max]['Title'].values[0]


In [ ]:
plt.figure(figsize=(10,10))
sns.lineplot(data=dbpop.groupby('Year')['runtime'].max())
plt.title('CUANTO DURA LA PELICULA POPULAR MAS LARGA DE CADA AÑO')
plt.ylabel('DURACIÓN (MINUTOS)')
plt.xlabel('AÑO')
plt.xticks(ticks=range(1904,2034,10))
plt.axhline(y=180, color='red', linestyle='dashdot')
plt.axhline(y=120, color='orange', linestyle='dashdot')
plt.axhline(y=240, color='maroon', linestyle='dotted')
plt.text(dbmaxyear, dbmax, f'{dbmaxtitle}, {int(dbmax)} min.', horizontalalignment='right')
plt.plot(dbmaxyear, dbmax, 'ro')
plt.text(dbminyear, dbmin, f'{dbmintitle}, {int(dbmin)} min.', horizontalalignment='right')
plt.plot(dbminyear, dbmin, 'ro')
plt.text(db2maxyear, db2max, f'{db2maxtitle}, {int(db2max)} min.', horizontalalignment='right')
plt.plot(db2maxyear, db2max, 'go') 
plt.text(db3maxyear, db3max, f'{db3maxtitle}, {int(db3max)} min.', horizontalalignment='right')
plt.plot(db3maxyear, db3max, 'bo')  
plt.show()


In [ ]:
dbpop.groupby('Year')['id'].count()

In [ ]:
plt.figure(figsize=(6,6))
sns.lineplot(dbpp.groupby('Year')['Title'].count())
plt.title('PELÍCULAS ANALIZADAS POR AÑO')
plt.ylabel('NÚMERO DE PELICULAS')
plt.xlabel('AÑO')
plt.xticks(ticks=range(1922,2032,10))
plt.show()

In [ ]:
dbesp['Year']=dbesp['Year'].replace(0,pd.NA)


In [ ]:
dbesp['Year'].value_counts().sort_index().index

In [ ]:
plt.figure(figsize=(7,7))
sns.lineplot(dbesp.groupby('Year')['runtime'].mean())
plt.title('DURACIÓN MEDIA DE LAS PELÍCULAS ESPAÑOLAS')
plt.ylabel('DURACIÓN (MINUTOS)')
plt.xlabel('Año')
plt.axhline(y=120, color='orange', linestyle='dashdot')
plt.xticks(ticks=range(1917,2037,10));

In [ ]:
duracion_media = dbesp.groupby('Year')['runtime'].mean()
num_peliculas = dbesp['Year'].value_counts().sort_index()

plt.figure(figsize=(7, 7))
sns.lineplot(x=duracion_media.index, y=duracion_media.values, label='Duración Media', color='blue')
plt.title('Duración Media de las Películas Españolas')
plt.ylabel('Duración Media (minutos)')
plt.xlabel('Año')
plt.xticks(ticks=range(1914, 2034, 10))
plt.legend(loc='upper left')

ax2 = plt.gca().twinx() 
sns.lineplot(x=num_peliculas.index, y=num_peliculas.values, label='Nº de Películas', ax=ax2, color='grey')
ax2.set_ylabel('Peliculas analizadas')
ax2.legend(loc='upper right')

# Mostrar el gráfico
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=moviesdb, x='budget', y='revenue')
plt.title('Presupuesto vs Recaudación')
plt.xlabel('Presupuesto')
plt.ylabel('Recaudación')
plt.xscale('log')
plt.yscale('log')
plt.show()



In [ ]:
sns.scatterplot(x=dbpop['revenue_x'],y=dbpop['runtime'],sizes=(10,500))


In [ ]:
sns.lineplot(movies.groupby('Year')['Runtime'].max())
sns.lineplot(movies22.groupby('Year')['Runtime'].max(),color='red')
plt.show()

Y tras todo esta limpieza y analisis de este ds, se ve que no hay nada apreciable. A ver si combinandolo con la otra ds consigo algo.

Voy a intentar rellenar los valores erroneos y los nans juntando todas

A ver si combinando los ds puedo rellenar algunos ceros, y de paso convertir los que no limpie en nans para que no me afecten al estudio.

El tamaño me permite entender que la fusion ha funcionado para alrededor de 4000 películas, de las 5200 que habia en movies_, buenos numeros.
Ahora voy a intentar rellenar nans y ceros de la original db, con las otras

Poco he limpiado, y aun habiendo convertido los 0 en nulls, sigue habiendo 0. Voy a añadir una columna de decadas para los tonos y colores en los graficos de correlaciones que voy a hacer.

In [ ]:
dbpop.shape

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(data=dbpop, x='budget_x', y='revenue_x', hue='decade_numeric', palette='magma', hue_norm=(dbpop['decade_numeric'].min(), dbpop['decade_numeric'].max()), alpha=0.5)
plt.title('PRESUPUESTO vs RECAUDACIÓN \n 4000 peliculas mas populares')
plt.xlabel('Presupuesto')
plt.ylabel('Recaudación')
plt.legend(title='Decada')
plt.xscale('log')
plt.yscale('log')
plt.xlim(60000,1000000000)
plt.ylim(60000,10000000000)

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=moviesdb_, x='budget_x', y='revenue_x',scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title('Presupuesto vs Recaudación (Regresion Lineal)')
plt.xlabel('Presupuesto')
plt.ylabel('Recaudación')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=dbpop, x='runtime', y='revenue_x', alpha=0.5)
plt.title('Relación entre la duración y la recaudación')
plt.xlabel('Duración (minutos)')
plt.ylabel('Recaudación ($)')
plt.yscale('log')
plt.show()

Necesito agrupar las duraciones como he hecho con las decadas

In [ ]:
grupos = [0, 90, 120, 150, 180, 240] 
label = ['<1.5h', '1.5-2h', '2-2.5h', '2.5-3h', '3h+']
dbpop.loc[:, 'runtime_group'] = pd.cut(dbpop['runtime'], bins=grupos, labels=label)


In [ ]:
dbpop.sort_values('budget_x',ascending=False)

Voy a tratar de mostrar la media de recaudacion de las peliculas agrupandolas por los intervalos de duración, para despues de ello hacer un gráfico que muestre las diferencias entre medias.

In [ ]:
runtimemedia = dbpop.groupby('runtime_group')['revenue_x'].mean()
runtimemed=runtimemedia.reset_index()
cuantos= dbpop.groupby('runtime_group').size()
runtimemed['contador'] = cuantos.values
runtimemed


In [ ]:
plt.figure(figsize=(10, 6))
bar=sns.barplot(data=runtimemed, x='runtime_group',y='revenue_x',color='skyblue')
plt.title('Recaudación promedio por duración')
plt.xlabel('Duración')
plt.ylabel('Recaudación promedio')
plt.bar_label(bar.containers[0], labels=runtimemed['contador'], label_type='edge', fontsize=10)

plt.ylim(10000000,500000000)
plt.show()

Aunque la gráfica tiene sentido, atendiendo al escaso numero de peliculas largas que hay pero el renombre y el impacto mediático que tiene cada una, voy a estudiar la rentabilidad por fragmentos, atendiendo al presupuesto y la recaudacion.

In [ ]:
dbpop.groupby('runtime_group')['budget_x'].mean()

In [ ]:
runtimebmedia = dbpop.groupby('runtime_group')['budget_x'].mean()
runtimebmed = runtimebmedia.reset_index()
cuantos = dbpop.groupby('runtime_group').size()
runtimebmed['contador'] = cuantos.values
runtimebmed

In [ ]:
runtimerent = dbpop.groupby('runtime_group').apply(lambda x: (x['revenue_x'] - x['budget_x']).mean())
runtimerentabilidad = runtimerent.reset_index(name='rentabilidad')
cuantos = dbpop.groupby('runtime_group').size()
runtimerentabilidad['contador'] = cuantos.values
print(runtimerentabilidad.columns);
runtimerentabilidad


In [ ]:
plt.figure(figsize=(10, 6))
bar=sns.barplot(data=runtimerentabilidad, x='runtime_group',y='rentabilidad',color='skyblue')
plt.title('Rentabilidad promedio por duración')
plt.xlabel('Duración')
plt.ylabel('Rentabilidad (recaudación-presupuesto)')
plt.bar_label(bar.containers[0], labels=runtimerentabilidad['contador'], label_type='edge', fontsize=10)
plt.gca().set_yticklabels([f'{int(y/1e6)}M' for y in plt.gca().get_yticks()]) #he tenido que buscar como evitar el 1e8 en la esquina


plt.ylim(60000000,400000000)
plt.show()

In [ ]:
peliculas_con_revenue_0 = dbpop[dbpop['revenue_x']<1000000]

ceroporgrupo = peliculas_con_revenue_0.groupby('runtime_group').size()
ceroporgrupo

Voy a intentar estudiar la hipotesis de las secuelas analizando la repeticion de palabras en los titulos. Aunque muchas secuelas o precuelas cambian el titulo o se dirigen a otro personaje de la pelicula original, hay un buen numero que simplemente añaden una coletilla o un numero a la pelicula original, o directamente copian el nombre (Aladdin, Scary Movie...)

In [ ]:
moviesdbinfo.rename(columns={'release_date':'Year'},inplace=True)
moviesdbinfo.rename(columns={'title':'Title'},inplace=True)
moviesdbinfo['popularity'] = moviesdbinfo['popularity'].apply(lambda x: str(x).split('.')[0])
moviesdbinfo['popularity'] = moviesdbinfo['popularity'].astype(int)
moviesdbinfo['Year'] = moviesdbinfo['Year'].apply(lambda x: str(x).split('-')[0])
moviesdbinfo['Year']=moviesdbinfo['Year'].replace('nan','0')
moviesdbinfo['Year']=moviesdbinfo['Year'].fillna(0)
moviesdbinfo['Year']=moviesdbinfo['Year'].replace(0,pd.NA)
moviesdbinfo['Year']=moviesdbinfo['Year'].astype('int')
moviesdb_['decade'] = (moviesdb_['Year'] // 10 * 10).astype(int).astype(str) + "s"
dbpop['decade'] = (dbpop['Year'] // 10 * 10).astype(str) + "s"
moviesdb_['decade_numeric'] = moviesdb_['Year'] // 10 * 10
dbpop['decade_numeric'] = dbpop['Year'] // 10 * 10

In [ ]:
moviesdbinfo['decade'] = (moviesdbinfo['Year'] // 10 * 10).astype(int).astype(str) + "s"
moviesdbinfo['decade_numeric'] = moviesdbinfo['Year'] // 10 * 10
moviesdbinfo.columns



In [ ]:
popinfo=moviesdbinfo[(moviesdbinfo['vote_count']>300)]
popinfo.shape

In [ ]:
palabrasec = ['sequel', 'remake', 'reboot', 'prequel', 'spin-off', 'spinoff', 'based on']
posiblesec = popinfo[popinfo['overview'].astype(str).str.contains('|'.join(palabrasec), case=False, na=False)]
posiblesec.shape

In [ ]:
posiblesec.head(2)

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Peliculas cuya descripcion incluye una palabra clave')
plt.ylabel('Nº de películas')
plt.xlabel('Decada')
sns.lineplot(posiblesec.groupby('decade')['Title'].count())

In [ ]:
print(popinfo[(popinfo['Year']>=2010)&(popinfo['Year']<2020)].shape)
print(popinfo[popinfo['Year']>=2020].shape)


In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(popinfo.groupby('decade_numeric')['Title'].count())
plt.title('Peliculas analizadas en la hipotesis de las secuelas')

Ya que con las descripciones no he obtenido los resultados que esperaba, aun teniendo en cuenta que hay tres veces mas peliculas de la anterior decada que de la actual, voy a intentarlo con los titulos, buscando las peliculas que repiten palabras grandes en sus nombres (ignorando los articulos y conectores comunes), y que incluyen números, numeros en romano, o 'Parte'. Para ello, admito que me he ayudado de ChatGPT pues no conseguia hacerlo bien para dividir los titulos en palabras y letras, buscar repeticiones y hacer la triple mascara junto con los numeros y las palabras clave.

In [ ]:
palabras_comunes = ['the', 'a', 'an', 'and', 'or', 'of', 'in', 'on', 'to', 'for', 'with', 'by']
palabras_clave = ['part', 'p.', 'volume', 'vol.']

# Filtrar títulos con palabras repetidas
mask_repetidos = popinfo['Title'].astype(str).str.lower().str.split().apply(lambda palabras: any(palabras.count(p) > 1 for p in palabras if p not in palabras_comunes))

# Filtrar con números o números romanos
mask_numeros = popinfo['Title'].str.contains(r'\b(?:[0-9]+|[ivxlcdmIVXLCDM]+)\b', na=False, case=False)

# Con palabras clave
mask_palabras_clave = popinfo['Title'].str.contains('|'.join(palabras_clave), case=False, na=False)

# Combinar todas las máscaras
masksecuelas = mask_repetidos | mask_numeros | mask_palabras_clave

titulossecuelas = popinfo[masksecuelas]

titulossecuelas.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Peliculas cuyo título incluye una palabra clave o repetida')
plt.ylabel('Nº de películas')
plt.xlabel('Decada')
sns.lineplot(titulossecuelas.groupby('decade')['Title'].count())

In [ ]:
secuelas=pd.merge(posiblesec,titulossecuelas,on=['Title','decade','decade_numeric'],how='outer')
secuelas.columns

In [ ]:
plt.figure(figsize=(8,8))
plt.title('Peliculas cuyo título o descripcion incluye una palabra clave o repetida')
plt.ylabel('Nº de películas')
plt.xlabel('Decada')
sns.lineplot(secuelas.groupby('decade')['Title'].count())

Tampoco me muestra nada cercano a mi hipótesis, asi que para terminar voy a tratar de conseguir un grafico que muestre el ratio de peliculas cuyo titulo o descripcion incluya una palabra clave/peliculas analizadas en esta hipotesis, por decadas, a ver si me aporta algo mas cercano a lo que creía.

In [ ]:
total_por_decada = popinfo.groupby('decade')['Title'].count()
secuelas_por_decada = secuelas.groupby('decade')['Title'].count()
ratio_por_decada = secuelas_por_decada / total_por_decada

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(ratio_por_decada.index,ratio_por_decada.values, color='skyblue')
plt.title('Ratio de películas con palabras clave en el título o descripción por Década')
plt.xlabel('Decada')
plt.ylabel('Ratio de películas')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

In [ ]:
plt.figure(figsize=(12,7))
plt.pie(data=conteo_generosnom,x=conteo_generosnom,labels=conteo_generosnom.values)
plt.title('Nominaciones a Oscar a Mejor Pelicula 1928-2020. \n (570 películas nominadas)')
plt.legend(conteo_generosnom.index,title='GÉNERO',loc='center left',bbox_to_anchor=(1,0.5))
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
plt.pie(data=conteo_generosw,x=conteo_generosw,labels=conteo_generosw.values)
plt.title('GANADORAS a Oscar a Mejor Película 1928-2020 \n -93 ediciones-',loc='center')
plt.legend(conteo_generosw.index,title='GÉNERO',loc='center left',bbox_to_anchor=(1,0.5))
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
sns.barplot(conteo_oscargenprincipal,color='gold')
plt.ylabel('PREMIOS')
plt.xlabel('GÉNERO PRINCIPAL')
plt.title('Oscar a Mejor Pelicula 1928-2020',loc='center')
plt.show()

In [ ]:
df = pd.read_csv("/Users/juanzubiaga/Desktop/BOOTCAMPDS/1-Local/EDA/J-EDA/TMDB_Movie_Limpito.csv")

In [ ]:
df.to_parquet("./test.parquet")

In [ ]:
df2 = pd.read_parquet("./test.parquet")

In [ ]:
df2.dtypes